In [2]:
#JSONとOAuth認証用のライブラリを使用
import json
from requests_oauthlib import OAuth1Session
import setting
import tweepy
import os

twitter = OAuth1Session(setting.CONSUMER_KEY, setting.CONSUMER_SECRET, setting.ACCESS_TOKEN, setting.ACCESS_SECRET)


In [3]:
# !pip install tweepy

In [23]:
keyword = "#バンキシャ"
q = keyword
#認証
auth = tweepy.OAuthHandler( setting.CONSUMER_KEY, setting.CONSUMER_SECRET )
auth.set_access_token( setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit = True)

#検索キーワード設定 
q = keyword

#つぶやきを格納するリスト
tweets_data =[]

#カーソルを使用してデータ取得
for tweet in tweepy.Cursor(api.search, q=q, count=100,tweet_mode='extended').items():

    #つぶやきテキスト(FULL)を取得
    tweets_data.append(tweet.full_text + '\n')

#出力ファイル名
dfile = "../data/hash_tag"
fname = "init.txt"
if not os.path.exists( dfile):
    os.mkdir(dfile)

#ファイル出力
with open(dfile+"/"+fname, "w",encoding="utf-8") as f:
    f.writelines(tweets_data)

# print
for i in range(len(tweets_data)):
    if i % 100 == 0: 
        print(tweets_data[i])

RT @mmmkkkmmm00: 絶対に絶対に違うんだけど、自分の心が腐っているので、桝アナの後ろのセットが中指立てているように見える。

#バンキシャ https://t.co/Z0zoHx61nq

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @bankisha: 夏目アナ解説】
感染拡大は"夜の人出"が引き金？ https://t.co/k1Hrrpqkge 

きのうの放送から夏目キャスターの解説パートをYouTube配信しています。
今後もどんどんアップしていきます
#バンキシャ #夏目美久 ＃新型コロナ

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @TONKO216: 番組冒頭から幸せそうな夏目さん
#夏目三久 #バンキシャ https://t.co/3WEyLViadE

【桝さん最高の司会でした☺️】 #バンキシャ #BangBang新番特番 @nittele_ntv

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

【マスミクコンビすき！】 #バンキシャ #BangBang新番特番 @nittele_ntv

【番組について一言】 #バンキシャ #BangBang新番特番 @nittele_ntv

RT @bankisha: 【バン犬と1分だけトーク】
放送後のとっておきトーク！
#古市憲寿 さんが 初登場 #伊沢拓司 さんに質問タイム！？
@poe1985 @tax_i_ 
#バンキシャ https://t.co/e9OGci44OJ

RT @com38682276: #バンキシャ
夏目さん幸せそうだよ🌟
良かったね🌟 https://t.co/rOfPmkDRKL

夏目さんてさ～、女性って感じより、女の子って感じがする～🤗

控えめで、可愛らしい感じだよね～🤗

それに、背が高いから、有吉さんきっと、理想の女性だったんやろ～ね😁

付き合うっていうより、もぉ～プロポーズしたんやろ～ね🤗

#バンキシャ 
#有吉弘行 
#夏目三久

RT @bankisha: ついに #桝太一 キャスターが加入
新

In [12]:
# pip install bert

In [16]:
# pip install transformers

In [20]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np
model = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
bert_tokenizer = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)

In [21]:
class BertForIMDb(nn.Module):
    '''BERTモデルにIMDbのポジ・ネガを判定する部分をつなげたモデル'''

    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()

        # BERTモジュール
        self.bert = net_bert  # BERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元、出力はポジ・ネガの2つ
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out